In [1]:
%load_ext autoreload
%autoreload 2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Flatten, LSTM
import pandas as pd
import numpy as np

In [2]:
from RainPredictionMachine.data import CleanDataRpm
cleaner = CleanDataRpm()
df = cleaner.clean_data()
df

,Chuva,Pres,Pres_max,Pres_min,Radiacao,Temp,Temp_orvalho,Temp_max,Temp_min,Temp_orvalho_max,...,Umid_min,Umid,Dir_vento,Rajada_vento,Vel_vento,Latitude,Longitude,Altitude,datahora,classe_chuva
0,0.0,948.6,948.6,947.6,0.0,24.7,21.1,25.6,24.6,21.1,...,74.0,80.0,82.0,3.3,1.1,-20.559167,-48.545,534.36,2020-01-01 00:00:00+00:00,0
1,0.0,949.7,949.8,948.7,0.0,23.4,20.7,24.8,23.4,21.0,...,78.0,84.0,12.0,3.2,0.8,-20.559167,-48.545,534.36,2020-01-01 01:00:00+00:00,0
2,0.0,949.8,950.2,949.7,0.0,23.7,20.0,23.9,23.4,20.8,...,79.0,80.0,338.0,3.5,1.0,-20.559167,-48.545,534.36,2020-01-01 02:00:00+00:00,0
3,0.6,949.8,949.9,949.5,0.0,23.0,20.0,23.7,23.0,20.0,...,79.0,83.0,44.0,3.5,1.1,-20.559167,-48.545,534.36,2020-01-01 03:00:00+00:00,1
4,0.2,949.2,949.8,949.2,0.0,22.3,21.6,22.9,22.0,22.4,...,83.0,96.0,358.0,3.5,1.3,-20.559167,-48.545,534.36,2020-01-01 04:00:00+00:00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,0.6,944.3,945.4,944.3,276.8,27.5,22.2,27.7,23.7,23.5,...,70.0,73.0,8.0,3.6,1.3,-20.559167,-48.545,534.36,2021-12-31 19:00:00+00:00,1
8756,0.0,944.1,944.3,944.0,139.6,27.7,22.0,28.6,26.6,22.8,...,63.0,71.0,121.0,3.1,0.5,-20.559167,-48.545,534.36,2021-12-31 20:00:00+00:00,0
8757,0.0,944.5,944.5,944.1,63.3,26.8,22.3,27.9,26.2,22.9,...,67.0,76.0,101.0,2.2,1.1,-20.559167,-48.545,534.36,2021-12-31 21:00:00+00:00,0
8758,0.0,944.8,944.9,944.5,14.0,24.6,21.5,26.8,24.6,22.3,...,74.0,83.0,129.0,4.4,1.4,-20.559167,-48.545,534.36,2021-12-31 22:00:00+00:00,0


In [3]:
df.columns

Index(['Chuva', 'Pres', 'Pres_max', 'Pres_min', 'Radiacao', 'Temp',
       'Temp_orvalho', 'Temp_max', 'Temp_min', 'Temp_orvalho_max',
       'Temp_orvalho_min', 'Umid_max', 'Umid_min', 'Umid', 'Dir_vento',
       'Rajada_vento', 'Vel_vento', 'Latitude', 'Longitude', 'Altitude',
       'datahora', 'classe_chuva'],
      dtype='object')

In [4]:
df.head(2).to_dict()

{'Chuva': {0: 0.0, 1: 0.0},
 'Pres': {0: 948.6, 1: 949.7},
 'Pres_max': {0: 948.6, 1: 949.8},
 'Pres_min': {0: 947.6, 1: 948.7},
 'Radiacao': {0: 0.0, 1: 0.0},
 'Temp': {0: 24.7, 1: 23.4},
 'Temp_orvalho': {0: 21.1, 1: 20.7},
 'Temp_max': {0: 25.6, 1: 24.8},
 'Temp_min': {0: 24.6, 1: 23.4},
 'Temp_orvalho_max': {0: 21.1, 1: 21.0},
 'Temp_orvalho_min': {0: 20.6, 1: 20.5},
 'Umid_max': {0: 80.0, 1: 84.0},
 'Umid_min': {0: 74.0, 1: 78.0},
 'Umid': {0: 80.0, 1: 84.0},
 'Dir_vento': {0: 82.0, 1: 12.0},
 'Rajada_vento': {0: 3.3, 1: 3.2},
 'Vel_vento': {0: 1.1, 1: 0.8},
 'Latitude': {0: -20.55916666, 1: -20.55916666},
 'Longitude': {0: -48.54499999, 1: -48.54499999},
 'Altitude': {0: 534.36, 1: 534.36},
 'datahora': {0: Timestamp('2020-01-01 00:00:00+0000', tz='UTC'),
  1: Timestamp('2020-01-01 01:00:00+0000', tz='UTC')},
 'classe_chuva': {0: 0, 1: 0}}

In [5]:
df.isnull().sum()

Chuva               0
Pres                0
Pres_max            0
Pres_min            0
Radiacao            0
Temp                0
Temp_orvalho        0
Temp_max            0
Temp_min            0
Temp_orvalho_max    0
Temp_orvalho_min    0
Umid_max            0
Umid_min            0
Umid                0
Dir_vento           0
Rajada_vento        0
Vel_vento           0
Latitude            0
Longitude           0
Altitude            0
datahora            0
classe_chuva        0
dtype: int64

In [6]:
def subsample_sequence(df, length):
    index = np.random.randint(0, df.shape[0] - length)
    df_sample = df.iloc[index:index+length]
    
    return df_sample

In [7]:
df_sample = subsample_sequence(df,48)
df_sample.head()

,Chuva,Pres,Pres_max,Pres_min,Radiacao,Temp,Temp_orvalho,Temp_max,Temp_min,Temp_orvalho_max,...,Umid_min,Umid,Dir_vento,Rajada_vento,Vel_vento,Latitude,Longitude,Altitude,datahora,classe_chuva
2154,0.2,948.1,948.7,948.1,1496.0,29.1,18.1,32.3,27.0,19.8,...,34.0,52.0,287.0,8.4,1.4,-20.559167,-48.545,534.36,2021-03-31 18:00:00+00:00,1
2155,0.0,947.4,948.1,947.4,1861.3,31.9,15.1,32.0,27.9,18.3,...,36.0,36.0,238.0,5.7,1.8,-20.559167,-48.545,534.36,2021-03-31 19:00:00+00:00,0
2156,0.0,947.2,947.5,947.2,1253.8,29.9,15.1,31.9,29.8,15.3,...,34.0,41.0,269.0,7.0,2.0,-20.559167,-48.545,534.36,2021-03-31 20:00:00+00:00,0
2157,0.0,947.4,947.4,947.1,228.5,27.6,15.9,29.9,27.4,16.0,...,40.0,49.0,230.0,5.5,1.6,-20.559167,-48.545,534.36,2021-03-31 21:00:00+00:00,0
2158,0.0,947.5,947.6,947.4,2.7,25.8,16.5,27.6,25.6,16.6,...,49.0,57.0,226.0,5.5,1.4,-20.559167,-48.545,534.36,2021-03-31 22:00:00+00:00,0


In [8]:
def split_subsample_sequence(df, length):
    '''Create one single random (X,y) pair'''
    
    df_subsample = subsample_sequence(df, length)
    y_sample = df_subsample['Chuva'].iloc[length - 24:]
    
    X_sample = df_subsample.drop(columns= ['classe_chuva', 'datahora'])[0:length -24]
    X_sample = X_sample.values
   
    return np.array(X_sample), np.array(y_sample)

In [9]:
def get_X_y(df, n_sequences, length):
    '''Return a list of samples (X, y)'''
    X, y = [], []

    for i in range(n_sequences):
        (xi, yi) = split_subsample_sequence(df, length)
        X.append(xi)
        y.append(yi)
        
    X = np.array(X)
    y = np.array(y)
    return X, y

In [10]:
train_size = int(df.shape[0]*0.8)
df_train = df.iloc[:train_size,:]
df_test = df.iloc[train_size:,:]

In [11]:
df_train.value_counts(['Chuva'])

Chuva
0.0      13162
0.2        290
0.4        104
0.1         54
0.6         50
         ...  
5.5          1
3.7          1
1.1          1
0.5          1
50.0         1
Length: 82, dtype: int64

In [12]:
df_test.shape

(3509, 22)

In [13]:
X_train,y_train = get_X_y(df_train,6000,72)

In [14]:
X_test,y_test = get_X_y(df_test,6000,72)

In [15]:
y_train.shape

(6000, 24)

In [16]:
# from sklearn.preprocessing import LabelEncoder
# from tensorflow.keras.utils import to_categorical

# # label = LabelEncoder()
# # y_enc = label.fit_transform(y.reshape((-1,)))
# y_cat = to_categorical(y_train)
# y_cat.shape

In [17]:
from tensorflow.keras.layers.experimental.preprocessing import Normalization

norm = Normalization()
norm.adapt(X_train)

2022-03-24 11:45:29.873529: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
X_train.shape

(6000, 48, 20)

In [19]:
model = Sequential()

model.add(norm)

model.add(LSTM(units=20, activation='tanh'))
model.add(Dense(10, activation="tanh"))
model.add(Dense(24, activation="relu"))

model.compile(loss='mse',
                  optimizer='rmsprop',
                  metrics=['mae','mse','mape'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, None, 20)         41        
 n)                                                              
                                                                 
 lstm (LSTM)                 (None, 20)                3280      
                                                                 
 dense (Dense)               (None, 10)                210       
                                                                 
 dense_1 (Dense)             (None, 24)                264       
                                                                 
Total params: 3,795
Trainable params: 3,754
Non-trainable params: 41
_________________________________________________________________


In [20]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=3, restore_best_weights=True)

model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1,
         validation_split=0.2,
          callbacks=[es])

Epoch 1/10
150/150 [==============================] - 2s 9ms/step - loss: 1.2012 - mae: 0.1741 - mse: 1.2012 - mape: 58109032.0000 - val_loss: 0.7994 - val_mae: 0.1628 - val_mse: 0.7994 - val_mape: 69931624.0000
Epoch 2/10
150/150 [==============================] - 1s 6ms/step - loss: 1.1923 - mae: 0.1845 - mse: 1.1923 - mape: 70319624.0000 - val_loss: 0.7962 - val_mae: 0.1583 - val_mse: 0.7962 - val_mape: 65813088.0000
Epoch 3/10
150/150 [==============================] - 1s 6ms/step - loss: 1.1874 - mae: 0.1844 - mse: 1.1874 - mape: 70826104.0000 - val_loss: 0.7948 - val_mae: 0.1676 - val_mse: 0.7948 - val_mape: 75897736.0000
Epoch 4/10
150/150 [==============================] - 1s 6ms/step - loss: 1.1836 - mae: 0.1866 - mse: 1.1836 - mape: 73482048.0000 - val_loss: 0.7931 - val_mae: 0.1610 - val_mse: 0.7931 - val_mape: 69120200.0000
Epoch 5/10
150/150 [==============================] - 1s 6ms/step - loss: 1.1804 - mae: 0.1844 - mse: 1.1804 - mape: 71471560.0000 - val_loss: 0.7917 - 

In [21]:
y_pred = model.predict(X_test)
y_pred

array([[0.1971819 , 0.09213358, 0.3059147 , ..., 0.4356676 , 0.33176988,
        0.3675969 ],
       [0.        , 0.00838355, 0.05791752, ..., 0.12350452, 0.10782351,
        0.11938658],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.202343  , 0.11012667, 0.34312907, ..., 0.42255044, 0.3462483 ,
        0.37716308],
       [0.6590021 , 0.71168375, 0.6389507 , ..., 0.41702342, 0.29552448,
        0.37253413],
       [0.84030694, 0.7881977 , 0.9603593 , ..., 0.28015622, 0.3856739 ,
        0.45777634]], dtype=float32)

In [22]:
model.evaluate(X_test,y_test)

188/188 [==============================] - 0s 2ms/step - loss: 0.4651 - mae: 0.2830 - mse: 0.4651 - mape: 137388992.0000


[0.4650806486606598, 0.28296488523483276, 0.4650806486606598, 137388992.0]

In [23]:
df_y_test = pd.DataFrame(y_test).T #cada coluna sera o indice para os dias do futuro -6000 dias

In [24]:
df_y_pred = pd.DataFrame(y_pred).T

In [25]:
pd.DataFrame(y_pred).describe()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
count,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,...,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000
mean,0.148234,0.139293,0.178726,0.264003,0.203486,0.000005,0.291347,0.154223,0.249036,0.282820,...,0.317519,0.245026,0.308925,0.431764,0.403649,0.424975,0.354166,0.340010,0.170578,0.236207
std,0.217793,0.173859,0.224765,0.243918,0.192982,0.000153,0.169638,0.101765,0.145427,0.140795,...,0.134224,0.151962,0.167534,0.254988,0.217231,0.262107,0.235184,0.183719,0.140422,0.143732
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.093098,0.095897,0.000000,0.201466,0.097660,0.152808,0.227182,...,0.247598,0.121129,0.186210,0.249081,0.261680,0.188918,0.186081,0.194252,0.055683,0.151241
50%,0.059496,0.107634,0.062124,0.245751,0.178794,0.000000,0.304148,0.139572,0.253741,0.337124,...,0.382524,0.259057,0.384743,0.449088,0.456966,0.451516,0.334415,0.393228,0.150385,0.242546
75%,0.192628,0.139319,0.319402,0.368923,0.258735,0.000000,0.435805,0.223747,0.363798,0.361801,...,0.408758,0.379583,0.437788,0.659363,0.590214,0.665198,0.561077,0.493084,0.289252,0.341908
max,1.302880,0.991138,1.093431,1.432445,1.062110,0.006897,0.743097,0.642801,0.658706,0.598933,...,0.626272,0.608945,0.625310,0.972682,0.779925,0.936390,0.914040,0.578938,0.628921,0.746143


In [26]:
def classe_chuva(precipitacao):
    mm=precipitacao
    if np.isnan(mm):
        chuva = "NaN"
    if mm == 0:
        chuva = 0 #'nao chove'
    elif mm >0 and mm <=5.0:
        chuva = 1 #'fraca'
    elif mm >5.0 and mm<=25.0:
        chuva = 2 #'moderada'
    else:
        chuva = 3 #'forte'
    return chuva

In [27]:
df_y_test_categorical = df_y_test.applymap(classe_chuva)
df_y_test_categorical.T.value_counts()

0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23
0  0  0  0  0  0  0  0  0  0  0   0   0   0   0   0   0   0   0   0   0   0   0   0     1459
                     1  1  1  1   1   1   1   1   1   1   1   1   1   0   0   0   0      170
1  0  0  0  0  0  0  0  0  0  0   0   1   1   1   1   1   1   1   1   1   1   1   1      153
   1  1  1  1  1  1  1  1  1  1   1   1   0   0   0   0   0   0   0   0   0   0   0      147
0  0  0  0  0  1  1  1  1  1  1   1   1   1   1   1   1   1   0   0   0   0   0   0      147
                                                                                        ... 
            2  0  0  0  0  0  0   0   1   1   0   0   1   0   0   0   0   0   0   0        1
                                      0   0   0   0   0   0   1   1   1   1   1   1        1
            1  1  1  0  0  0  0   0   0   0   1   1   0   0   0   0   1   0   0   0        1
                                              0   0   0   0   0   0   0   0   

In [28]:
df_y_pred_categorical = df_y_pred.applymap(classe_chuva)
df_y_pred_categorical.T.value_counts()

0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23
1  1  1  1  1  0  1  1  1  1  1   1   1   1   1   1   1   1   1   1   1   1   1   1     2626
      0  1  1  0  1  1  1  1  1   1   1   1   1   1   1   1   1   1   1   1   1   1      551
0  1  0  1  1  0  1  1  1  1  1   1   1   1   1   1   1   1   1   1   1   1   1   1      511
1  1  1  1  1  0  1  1  1  1  1   1   1   0   1   1   1   1   1   1   1   1   1   1      230
0  0  0  1  1  0  1  1  1  1  1   1   1   1   1   1   1   1   1   1   1   1   0   1       83
                                                                                        ... 
         0  0  0  1  0  0  0  1   0   1   0   0   0   0   0   0   0   0   1   0   0        1
   1  1  1  1  0  1  1  1  1  1   1   1   0   1   0   1   1   1   0   1   1   1   1        1
   0  0  0  0  0  0  1  1  1  1   1   1   1   1   1   1   1   1   1   1   1   0   0        1
   1  1  1  1  0  1  1  1  1  1   1   1   0   1   1   1   1   1   1   1   1   

In [29]:
(df_y_test_categorical == df_y_pred_categorical).sum() # o quanto acertamos por dia

0       14
1        5
2       23
3       12
4       14
        ..
5995    11
5996    14
5997    14
5998     4
5999     2
Length: 6000, dtype: int64

In [30]:
(df_y_test_categorical == df_y_pred_categorical).sum().sum()/(6000*24) #accuracy

0.46627083333333336